# Darcy Flow
Including forward and inverse problem solver for 1D. Darcy Flow in 

Darcy Flow
\begin{equation}
    \nabla \cdot ( k(x) \nabla u(x) ) = f(x)
\end{equation}

Darcy Flow in 1D
\begin{equation}
    (k(x)u'(x))' = k'(x) u'(x) + k(x) u''(x) = f(x)
\end{equation}

## Conditions for the problem
Zero Dirichlet boundary conditions $u(0) = u(1) = 0$

## Hints

Express $u$ as $u(x) = \sum_{j = 1}^{n} u_{j} \phi_{j}(x)$ with $\phi_{j}(x) =$ piecewise linear tent function peaking at node $j$

Solve $A(u_{1}, …, u_{n}) = b$ for the coefficients  
\begin{align}
b_{j} &= \int_{0}^{1} f(x) \phi_{j}(x) d x \\
A_{i j} &= \int_{0}^{1} k(x) \phi'_{i}(x) \phi'_{j}(x) d x
\end{align}
Google finite element method / Galerkin method for elliptic PDE 

I will give you e.g. (u(1/4), u(1/2), u(3/4)) and similarly for f, both corrupted by additive N(0, \sigma^{2}) noise.
Your challenge:  infer k

--

Modelling assumption:  $k(x) = exp( \sum_{\alpha = 0}^{A} k_{\alpha} \phi_{\alpha}(x) )$

--

Try a Fourier basis. Note that u vanishes at the boundary, but that doesn't mean k does.

Create the forward model. Solver works. Extend to imitation inverse model with likelihood, etc. Use this to solve his problem.

## Useful Links
http://www.mathematik.uni-dortmund.de/~kuzmin/Transport.pdf  
https://en.wikipedia.org/wiki/Finite_element_method

# The Problem
\begin{align}
    k'(x) u'(x) + k(x) u''(x) &= f(x) \\
    u(0) = u(1) &= 0
\end{align}

We put the equation into the weak form. $v(x)$ is a function which satisfies the boundary conditions.

\begin{equation}
    \int_{0}^{1} f(x)v(x) \ dx = \int_{0}^{1} (k'(x) u'(x) v(x) + k(x) u''(x) v(x)) \ dx
\end{equation}

We can integrate by parts and apply our boundry conditions on the second L.H.S. term to arrive at the next equation.

\begin{align}
    \int_{0}^{1} f(x)v(x) \ dx &= k(x) u'(x) v(x)|_{0}^{1} + \int_{0}^{1} (k'(x) u'(x) v(x) - k(x) u'(x) v'(x)) \ dx \\
    \int_{0}^{1} f(x)v(x) \ dx &= \int_{0}^{1} (k'(x) u'(x) v(x) - k(x) u'(x) v'(x)) \ dx
\end{align}

We choose the piecewise linear function $v_k$ for a discretization.

\begin{equation}
v_{k}(x) = 
    \begin{cases}
        \frac{x-x_{k-1}}{x_k-x_{k-1}} & \text{if } x \in [x_{k-1}, x_k] \\
        \frac{x_{k+1}-x}{x_{k+1}-x_k} & \text{if } x \in [x_{k}, x_{k+1}] \\
        0 & \text{otherwise}
    \end{cases}
\end{equation}

If we expand $u(x)$ in a basis of tent functions on this discretization, we are left with the problem

\begin{equation}
    A \bf{u} = \bf{b}
\end{equation}

where $A_{ij} = \int (k'(x) v_{i}'(x) v_{j}(x) - k(x) v_{i}'(x) v_{j}'(x)) \ dx$ and $b_{j} = \int f(x) v_{j} dx$.